In [1]:
import sys
sys.path.append('../')

import sqlite3
import pandas as pd
from src.cmesrc.config import CMESRCV2_DB, SPATIOTEMPORAL_MATCHING_HARPS_DATABASE_PICKLE, DIMMINGS_MATCHED_TO_HARPS_PICKLE

In [2]:
conn = sqlite3.connect(CMESRCV2_DB)
conn.execute("PRAGMA foreign_keys = OFF")
cur = conn.cursor()

# Move the table "CMES_HARPS_SPATIALLY_CONSIST" to name "OLD_CMES_HARPS_SPATIALLY_CONSIST"

# Check if "OLD_CMES_HARPS_SPATIALLY_CONSIST" exists
cur.execute("""
            SELECT name FROM sqlite_master WHERE type='table' AND name='OLD_CMES_HARPS_SPATIALLY_CONSIST'
			"""
)

if len(cur.fetchall()) == 0:
	print("OLD_CMES_HARPS_SPATIALLY_CONSIST does not exist. Creating it now.")
	cur.execute("""
 				ALTER TABLE CMES_HARPS_SPATIALLY_CONSIST
 				RENAME TO OLD_CMES_HARPS_SPATIALLY_CONSIST
 				""")
else:
	choice = input("OLD_CMES_HARPS_SPATIALLY_CONSIST already exists. Replace it? (y/n)")
	if choice == 'y':
		cur.execute("""
					DROP TABLE OLD_CMES_HARPS_SPATIALLY_CONSIST
					""")
		cur.execute("""
					ALTER TABLE CMES_HARPS_SPATIALLY_CONSIST
					RENAME TO OLD_CMES_HARPS_SPATIALLY_CONSIST
					""")

		print("OLD_CMES_HARPS_SPATIALLY_CONSIST replaced.")
	else:
		new_choice = input("Drop CMES_HARPS_SPATIALLY_CONSIST and replace it? (y/n)")
		
		if new_choice == "y":
			cur.execute("""
						DROP TABLE CMES_HARPS_SPATIALLY_CONSIST
						""")
			print("CMES_HARPS_SPATIALLY_CONSIST dropped.")
		else:
			raise Exception("Cannot proceed without dropping CMES_HARPS_SPATIALLY_CONSIST or overwriting OLD_CMES_HARPS_SPATIALLY_CONSIST")

# Create a new table "CMES_HARPS_SPATIALLY_CONSIST" with the same schema as the old one

cur.execute("""
CREATE TABLE "CMES_HARPS_SPATIALLY_CONSIST" (
	"harpnum"	INTEGER,
	"cme_id"	INTEGER,
	PRIMARY KEY("harpnum","cme_id"),
	FOREIGN KEY("harpnum") REFERENCES "HARPS"("harpnum"),
	FOREIGN KEY("cme_id") REFERENCES "CMES"("cme_id")
);
""")

CMES_HARPS_SPATIALLY_CONSIST dropped.


In [3]:

df = pd.read_pickle(SPATIOTEMPORAL_MATCHING_HARPS_DATABASE_PICKLE)
df = df[df["HARPS_SPAT_CONSIST"]]

for cme_id, harpnum in df[["CME_ID", "HARPNUM"]].values:
    cme_id = int(cme_id[2:])
    harpnum = int(harpnum)

    # Add the match to CMES_HARPS_SPATIALLY_CONSIST

    cur.execute("""
                INSERT INTO CMES_HARPS_SPATIALLY_CONSIST (harpnum, cme_id)
                VALUES (?, ?)
                """, (harpnum, cme_id))

In [4]:
conn.commit()

In [5]:
# Same with dimmings now

# Move the table "DIMMINGS" to name "OLD_DIMMINGS"

# Check if "OLD_DIMMINGS" exists
cur.execute("""
            SELECT name FROM sqlite_master WHERE type='table' AND name='OLD_DIMMINGS'
            """
)

if len(cur.fetchall()) == 0:
    print("OLD_DIMMINGS does not exist. Creating it now.")
    cur.execute("""
                    ALTER TABLE DIMMINGS
                    RENAME TO OLD_DIMMINGS
                    """)
else:
    choice = input("OLD_DIMMINGS already exists. Replace it? (y/n)")
    if choice == 'y':
        cur.execute("""
                        DROP TABLE OLD_DIMMINGS
                        """)
        cur.execute("""
                        ALTER TABLE DIMMINGS
                        RENAME TO OLD_DIMMINGS
                        """)

        print("OLD_DIMMINGS replaced.")
    else:
        new_choice = input("Drop DIMMINGS and replace it? (y/n)")

        if new_choice == "y":
            cur.execute("""
                            DROP TABLE DIMMINGS
                            """)
            print("DIMMINGS dropped.")
        else:
            raise Exception("Cannot proceed without dropping DIMMINGS or overwriting OLD_DIMMINGS")

        
cur.execute("""
CREATE TABLE "DIMMINGS" (
	"dimming_id"	INTEGER NOT NULL,
	"harpnum"	INTEGER,
	"harps_dimming_dist"	REAL NOT NULL,
	"dimming_start_date"	TEXT NOT NULL,
	"dimming_peak_date"	TEXT NOT NULL,
	"dimming_lon"	REAL NOT NULL,
	"dimming_lat"	REAL NOT NULL,
	"image_timestamp"	TEXT,
	PRIMARY KEY("dimming_id"),
	FOREIGN KEY("image_timestamp") REFERENCES "IMAGES"("timestamp"),
	FOREIGN KEY("harpnum") REFERENCES "HARPS"("harpnum")
);
""")

DIMMINGS dropped.


In [6]:
dimmings = pd.read_pickle(DIMMINGS_MATCHED_TO_HARPS_PICKLE)
dimmings = dimmings[dimmings["MATCH"]]

for i, row in dimmings[dimmings["MATCH"]].iterrows():
    cur.execute(
        """
        INSERT INTO dimmings (dimming_id, harpnum, harps_dimming_dist, dimming_start_date, dimming_peak_date, dimming_lon, dimming_lat)
        VALUES (?, ?, ?, ?, ?, ?, ?)
        """,
        (
            int(row["dimming_id"]),
            int(row["HARPNUM"]),
            row["HARPS_DIMMING_DISTANCE"],
            row["start_time"].split(".")[0],
            row["max_detection_time"].iso[:-4],
            row["longitude"],
            row["latitude"]
        )
    )

In [7]:
conn.commit()
conn.close()

In [8]:
from datetime import datetime
from tqdm import tqdm
import bisect

# Function to find the closest timestamp using binary search
def closest_timestamp(target, sorted_timestamps):
    index = bisect.bisect_left(sorted_timestamps, target)
    if index == 0:
        return sorted_timestamps[0]
    if index == len(sorted_timestamps):
        return sorted_timestamps[-1]
    before = sorted_timestamps[index - 1]
    after = sorted_timestamps[index]
    if after - target < target - before:
       return after
    else:
       return before

conn = sqlite3.connect(CMESRCV2_DB)
cur = conn.cursor()

image_timestamps = cur.execute("SELECT timestamp FROM images").fetchall()
image_timestamps = [t[0] for t in image_timestamps]

timestamps = sorted([datetime.strptime(t, "%Y-%m-%d %H:%M:%S") for t in image_timestamps])
image_timestamps = timestamps

for row in tqdm(cur.execute("SELECT dimming_id, dimming_start_date FROM dimmings").fetchall()):
    dimming_timestamp = datetime.strptime(row[1], "%Y-%m-%d %H:%M:%S")
    image_timestamp = closest_timestamp(dimming_timestamp, image_timestamps)
    cur.execute("UPDATE dimmings SET image_timestamp = ? WHERE dimming_id = ?", (image_timestamp.strftime("%Y-%m-%d %H:%M:%S"), row[0]))

conn.commit()
conn.close()

100%|██████████| 2514/2514 [00:00<00:00, 117331.67it/s]
